I am going to use the following metric functions
1. closeness
2. degree
3. betweenness

In [1]:
import preprocessing as ps
import numpy as np
import networkx as nx
import pickle
import matplotlib.pyplot as plt
import random
random.seed(42)
import csv
import algos as al
import metrics as me

In [2]:
# create the project networks
list_1 = [('DM', 'T'), ('DB', 'T'), ('AI', 'T')]
list_2 = [('DM', 'T'), ('DB', 'T'), ('AI', 'T'), ('DM', 'DB'), ('DM', 'AI'), ('DB', 'AI')]
list_3 = [('DM', 'T'), ('DB', 'T'), ('AI', 'DB')]

star_proj = ps.createProjectNetwork(list_1)
full_proj = ps.createProjectNetwork(list_2)
chain_proj = ps.createProjectNetwork(list_3)

main_network = pickle.load(open(f"./networks/new_subnets/10_nodes.pkl", 'rb'))
project_net = star_proj

In [3]:
projects = [(star_proj, "Star-Structured Project"), 
            (full_proj, "Fully-connected Structured Project"), 
            (chain_proj, "Chain-Structured Project")]

beta = None
obj_max_2 = 0.0
best_set_2 = set()

metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
print("Influential First")
influential_nodes = al.get_top_node_from_each_group(network_main.copy(), project_net, metric_fn)
for node in influential_nodes:
    subset, comm_eff = al.Greedy(network_main.copy(), project_net, node, metric_fn=metric_fn, beta=beta)
    if comm_eff > obj_max_2:
        obj_max_2 = comm_eff
        best_set_2 = subset

total_inf = sum(al.leader_eff(network_main.copy(), project_net, metric_fn, node, beta=beta) for node in best_set_2)
print("Coordinators communication efficiency", total_inf)
# print(f"Objective value : {obj_max_2}")

print("Intra-team ranking")
for node in list(best_set_2):
    print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.intra_team_rank(network_main, metric_fn, node)}")

print("\n")

print("Inter-team ranking")
for node in list(best_set_2):
    print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.inter_team_rank(network_main, project_net, metric_fn, node)}")

Influential First
Coordinators communication efficiency 581.0647364331573
Intra-team ranking
Team :AI, Node: Eiji Takimoto, Rank: 1
Team :DB, Node: Yosihisa Udagawa, Rank: 1
Team :T, Node: F. Bruce Shepherd, Rank: 1
Team :DM, Node: Bing Liu, Rank: 8


Inter-team ranking
Team :AI, Node: Eiji Takimoto, Rank: 1
Team :DB, Node: Yosihisa Udagawa, Rank: 6
Team :T, Node: F. Bruce Shepherd, Rank: 14
Team :DM, Node: Bing Liu, Rank: 2


In [5]:
projects = [(star_proj, "Star-Structured Project"), 
            (full_proj, "Fully-connected Structured Project"), 
            (chain_proj, "Chain-Structured Project")]

beta = None
obj_max_2 = 0.0
best_set_2 = set()

metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
print("Greedy Algorithm")
for node in network_main.nodes():
    subset, comm_eff = al.Greedy(network_main.copy(), project_net, node, metric_fn=metric_fn, beta=beta)
    if comm_eff > obj_max_2:
        obj_max_2 = comm_eff
        best_set_2 = subset

total_inf = sum(al.leader_eff(network_main.copy(), project_net, metric_fn, node, beta=beta) for node in best_set_2)
print("Coordinators communication efficiency", total_inf)
# print(f"Objective value : {obj_max_2}")

print("Intra-team ranking")
for node in list(best_set_2):
    print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.intra_team_rank(network_main, metric_fn, node)}")

print("\n")

print("Inter-team ranking")
for node in list(best_set_2):
    print(f"Team :{network_main.copy().nodes[node]['label']}, Node: {node}, Rank: {al.inter_team_rank(network_main, project_net, metric_fn, node)}")

Greedy Algorithm
Coordinators communication efficiency 581.0647364331575
Intra-team ranking
Team :AI, Node: Eiji Takimoto, Rank: 1
Team :DM, Node: Bing Liu, Rank: 8
Team :T, Node: F. Bruce Shepherd, Rank: 1
Team :DB, Node: Toshimi Minoura, Rank: 2


Inter-team ranking
Team :AI, Node: Eiji Takimoto, Rank: 1
Team :DM, Node: Bing Liu, Rank: 2
Team :T, Node: F. Bruce Shepherd, Rank: 14
Team :DB, Node: Toshimi Minoura, Rank: 1


In [6]:
# Using Random Strategy
projects = [(star_proj, "Star-Structured Project"), 
            (full_proj, "Fully-connected Structured Project"),
            (chain_proj, "Chain-Structured Project")]

num_iter = 1000
metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
al.randomMonteCarlo(network_main, project_net, metric_fn, num_iter)

773.7

In [7]:
projects = [(star_proj, "Star-Structured Project"), 
            (full_proj, "Fully-connected Structured Project"),
            (chain_proj, "Chain-Structured Project")]

metric_fn = me.compute_closeness_centrality
network_main = main_network.copy()
al.inteam_influence_only(network_main, project_net, metric_fn)

Intra-team ranking
Team :DM, Node: Tianbing Xu, Rank: 1
Team :DB, Node: Yosihisa Udagawa, Rank: 1
Team :T, Node: F. Bruce Shepherd, Rank: 1
Team :AI, Node: Eiji Takimoto, Rank: 1


Inter-team ranking
Team :DM, Node: Tianbing Xu, Rank: 9
Team :DB, Node: Yosihisa Udagawa, Rank: 6
Team :T, Node: F. Bruce Shepherd, Rank: 14
Team :AI, Node: Eiji Takimoto, Rank: 1


567.6750253690278